### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 3. Cosets: Enumerating Complex Reflection Groups

![Numbers](images/numbers.jpg)

## Setup

First, reload the algorithms from earlier ...

In [ ]:
LoadPackage("jupyterviz");
opts := rec(vertexwidth := 12, vertexheight := 12, edgecolor := "#def");;
Read("orbits.g");
Read("variants.g");
Read("examples.g");

## Complex Reflection Groups ...

... don't have
* a well-behaved length function
* simple reflections as generators
* root systems
* ...

But they do have
* a reflection representation
* parabolic subgroups
* ...
* a nice (Coxeter-like) presentation

Questions:
* how to construct the reflection representation systematically?
* how to turn the presentation into a usable group?

Here, we will only address the latter question, in the form of the **Todd-Coxeter coset enumeration** procedure,
a further variant of the orbit algorithm which potentially converts the group presentation into a finite permutation group. 

## Data Nodes

* Recall all the modified orbit algorithms.
* Here is all in one:  words, transversal and images.

In [ ]:
orbit_and_more := function(aaa, x, under)
    local   list,  words,  reps,  images,  i,  k,  z,  l;
    list := [x];  words := [[]];  reps := [aaa[1]^0];
    images := List(aaa, x -> []);  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                Add(words, onWords(words[i], k));
                Add(reps, reps[i] * aaa[k]);
                l := Length(list);
            fi;
            images[k][i] := l;
        od;
    od;
    return rec(list := list, words := words, reps := reps, images := images);
end;

* for example

In [ ]:
orb := orbit_and_more(transpositions(5), [1,2], OnSets);

In [ ]:
edges := Union(List(orb.images, list_with_index));
PlotGraph(Filtered(edges, x-> x[1] <> x[2]), opts);

* Managing all these property lists in parallel is a bit unwieldy, and perhaps confusing in the long run ...
* Let's transpose the setup and store properties as data with each node.

### Data Type

* An `Item` is a node object with **attributes**
  * `key`: an element of the domain that is acted upon
  * `idx`: its position in the list containing the orbit
  * `next`: a list of child nodes, one for each generator
  * `data`: other useful information

In [ ]:
ItemFamily := NewFamily("ItemFamily", IsObject);

DeclareRepresentation("IsItem",
    IsComponentObjectRep and IsAttributeStoringRep,
    ["key", "idx", "data", "next"]
);

ItemType := NewType(ItemFamily, IsItem);

### Constructor

* An `Item` object is constructed from a key $x \in X$.


In [ ]:
Item := function(key)
    local   r;
    r := rec(key := key, data := rec(), next := []);
    return Objectify(ItemType, r);
end;

In [ ]:
item := Item([1,2]);

### Printing

* It will be convenient to install a method that prints a meaningful representation of an `Item` object.

In [ ]:
InstallMethod(PrintObj, "for items", true, [IsItem], 0, function(item)
    Print("Item( ", item!.key, " )");
end);

In [ ]:
item;

###  Comparison

* Items are compared with respect to their keys, for equality, and for size.

In [ ]:
InstallMethod(\=, "for items", true, [IsItem, IsItem], 0, function(itemL, itemR)
    return itemL!.key = itemR!.key;
end);

InstallMethod(\<, "for items", true, [IsItem, IsItem], 0, function(itemL, itemR)
    return itemL!.key < itemR!.key;
end);

In [ ]:
item = item;
item < item;
item <= item;
Set([item, item]);

### Data Orbits

* We can now reformulate the above omnibus orbit algorithm in terms of such items.

In [ ]:
orbit_with_data := function(aaa, item, under)
    local   list,  x,  k,  a,  y,  z;
    list := [item];  item!.idx := 1;  
    item!.data := rec(rep := (), word := []);
    for x in list do
        for k in [1..Length(aaa)] do
            a := aaa[k];
            y := Item(under(x!.key, a));
            z := First(list, z -> z = y);
            if z = fail then
                Add(list, y);  y!.idx := Length(list);
                y!.data := rec(  
                  rep := x!.data.rep * a,
                  word := onWords(x!.data.word, k),
                );
                z := y;
            fi;
            x!.next[k] := z!.idx;
        od;
    od;
    return list;
end;

In [ ]:
orb := orbit_with_data(transpositions(5), item, OnSets);

In [ ]:
List(orb, x-> x!.idx);
List(orb, x-> x!.data);

In [ ]:
edges := Union(List(orb, o-> List(o!.next, t-> [o!.idx, t])));
PlotGraph(Filtered(edges, x-> x[1] <> x[2]), opts);

## Example

* The complex reflection group $G_{12}$ has a presentation 
$$
\langle
s_1, s_2, s_3 \mid
s_1^2 = s_2^2 = s_3^2 = 1,\,
s_1 s_2 s_3 s_1 = s_2 s_3 s_1 s_2 = s_3 s_1 s_2 s_3
\rangle
$$
* Let's try and enumerate its elements systematically.

## Smart Nodes

* We will use a similar data structure, `Node`, for the purpose of coset enumeration.
* Here, the `idx` attribute is used to identify `Node` objects.
* And a `data` attribute is shared between all `Node` objects.

In [ ]:
NodeFamily := NewFamily("NodeFamily", IsObject);

DeclareRepresentation("IsNode",
    IsComponentObjectRep and IsAttributeStoringRep, 
    ["idx", "word", "data", "next"]
);

NodeType := NewType(NodeFamily, IsNode);

In [ ]:
Node := function(word, data)
    local   node;
    node := Objectify(NodeType, rec(word := word, data := data, next := []));
    Add(data.list, node);  node!.idx := Length(data.list);
    data.active := data.active + 1;
    return node;
end;

### Print

In [ ]:
InstallMethod(PrintObj, "for nodes", true, [IsNode], 0, function(node)
    Print("Node( ", node!.idx, " )");
end);
InstallMethod(String, "for nodes", true, [IsNode], 0, function(node)
    return Concatenation("Node( ", String(node!.idx), " )");
end);

In [ ]:
data := rec(list := [], active := 0);
node := Node([], data);

### Comparison

In [ ]:
InstallMethod(\=, "for nodes", true, [IsNode, IsNode], 0, function(nodeL, nodeR)
    return nodeL!.idx = nodeR!.idx;
end);

InstallMethod(\<, "for nodes", true, [IsNode, IsNode], 0, function(nodeL, nodeR)
    return nodeL!.idx < nodeR!.idx;
end);

In [ ]:
node = node;
node < node;
node <= node;
node > node;

## Coset Enumeration

* Q: What is $G = \langle S \mid R \rangle$?
* A: Todd-Coxeter!

* Suppose that a group $G$ is given by a **presentation** $\langle S \mid R \rangle$, consisting of a (finite) set $S$ of abstract **generators** $s_1, s_2, \dots, s_k$, and a (finite) list $R$ of **relations** $l_j = r_j$, where both $l_j$ and $r_j$ are words in $S \cup S^{-1}$.

* For convenience, we assume that $S$ is closed under inverses: $S  = S^{-1}$.

* We wish to enumerate the elements of $G$ (hoping that $G$ is a finite group), or more generally, the cosets of a subgroup $H$ of $G$ (hoping that $H$ has finite index in $G$).

* A priori, neither the domain $X$ being acted upon (by $G$), nor the edges of the action graph are known.

* Strategy: define new nodes as images of old nodes under a generator, but be prepared to identify this node with an existing one, if the relations imply they are the same.

* For this, each `Node` object $x$ has
  * a unique ID `idx` (where `idx` $ = n \iff x = x_n$),
  * a word `word` $ \in S^*$ (corresponding to a path in the BFS spanning tree of the action graph),
  * images $x$.`next`$[s] = x.s$ for each $s \in S$ (where $x.s \in X \cup \{ \perp \}$)
  * a reference $x$.`flat`$ \in X \cup \{ \perp \}$ to the node it has possibly been replaced by.
  
* Eventually, we want that $x.s \in X$ for all $x \in X$, $s \in S$.

###  Flatness

* A node $x$ is **active** if $x$.`flat`$ = {\perp}$.

In [ ]:
isActive := node -> not IsBound(node!.flat);

In [ ]:
isActive(node);

In [ ]:
node!.flat := 0;;
isActive(node);

In [ ]:
Unbind(node!.flat);
isActive(node);

* Each node $x \in X$ has an associated active node $x^{\flat}$ defined recursively as
$$
x^{\flat} = \begin{cases}
x, & \text{if } x.\text{flat} = {\perp}\\
(x.\text{flat})^{\flat}, & \text{else}
\end{cases}
$$

In [ ]:
flat := function(node)
    while IsBound(node!.flat) do  node := node!.flat;  od;
    return node;
end;

### Images

* Recall that $S^{-1} = S$.  Assume that `data.invr` holds the map $s \mapsto s^{-1}$.
* In words, we write $-s$ for $s^{-1}$.  
* So to find $x.s$ for $s \in S = S^{-1}$ we need to replace $s$ by `data.invr`$[-s]$ first, if $s < 0$.

In [ ]:
getImage := function(node, s)
    if s < 0 then  s := node!.data.invr[-s];  fi;
    return GetWithDefault(node!.next, s, false);
end;

* To sprout a new node $x.s$:

In [ ]:
sprout := function(node, s)
    local   next;
    next := Node(onWords(node!.word, s), node!.data);
    node!.next[s] := next;
    next!.next[node!.data.invr[s]] := node;
    return next;
end;

### Actions

* We will work with two distinct actions:
  * a **partial action** which returns `false` if an image does not exist (yet)
  * a **sprouting action** which sprouts a new node if necessary.

In [ ]:
onNodesPartial := function(node, s)
    local   next;
    next := getImage(node, s);
    if next = false then  return false;  fi;
    return flat(next);
end;

In [ ]:
onNodesSprout := function(node, s)
    local   next;
    next := getImage(node, s);
    if next = false then 
        return sprout(node, s);
    else
        return flat(next);  
    fi;
end;

* Both actions need only be defined on the generators $s \in S$, and can then be applied to words in $S^*$.

In [ ]:
nodeUnderWordSprout := function(node, word)
    local   s;
    for s in word do
        node := onNodesSprout(node, s);
    od;
    return node;
end;

In [ ]:
nodeUnderWordPartial := function(node, word)
    local   s;
    for s in word do
        node := onNodesPartial(node, s);
        if node = false then  return node;  fi;
    od;
    return node;
end;

### Enumerate!

* We now formulate the `enumerate` procedure which takes a presentation `genrel` for a group $G$ as input and produces a permutation group as output.  Specifically, `genrel` has components
  * `gens`: a list `[1..n]` of abstract generators $S = S^{-1}$
  * `rels`: a list of relations expressed as pairs of word in $S$
  * `invr`: the map $S \to S: s \mapsto s^{-1}$
  * `sbgp`: a subset of $S$, generating a subgroup $H$ of $G$.

In [ ]:
enumerate := function(genrel)
    local  data,  node,  word,  s;

    # initialize.
    data := rec(list := [], active := 0);
    data.invr := genrel.invr;
    data.variants := VariantsRelations(genrel);
    node := Node([], data);
    
    # first close the subgroup tables.
    for word in genrel.sbgp do
        trace(node, word);
    od;
    
    # process nodes in the queue
    for node in data.list do
        for s in genrel.gens do
            process(node, s);
        od;
    od;

    # return data
    return data;
end;

###  Tracing Words

* To trace a node $x$ under a word $w$ means to make sure that $x.w = x$, using the sprouting action.
* If $w \in H$ then $x_1.w = x_1$ should hold.
* If $l = r$ is a relation then $w:= l/r = 1$ and $x.w = x$ should hold for any $x \in X$.
* In any case, for the last letter of $w$, we carefully check if the resulting coset is already known or not.

In [ ]:
trace := function(node, word)
    local   other;
    other := nodeUnderWordSprout(node, word{[1..Length(word)-1]});
    updateEdge(other, word[Length(word)], node);
end;

### Processing a Node under a Generator

* To find $x.s$, use variants of the relations to express $s$ as a word $w$ in the generators and check if $x.w$ is determined already.  If so, carefully set $x.s$ to $s.w$.  If this doesn't work out, create a new node $x.s$.

In [ ]:
process := function(node, s)
    local   variant,  next;
    for variant in node!.data.variants[s] do
        if isActive(node) then
            next := nodeUnderWordPartial(node, variant);
            if next <> false then  updateEdge(node, s, next);  fi;
        fi;
    od;
    if isActive(node) and not IsBound(node!.next[s]) then sprout(node, s); fi;
end;

### Edges

* In the (directed) .graph of a group action, an edge $x \stackrel{s}{\longrightarrow} y$ always comes with the opposite edge $y \stackrel{s^{-1}}{\longrightarrow} x$.
* Thus, carefully updating $x.s = y$ always refers to two edges of the graph.

In [ ]:
updateEdge := function(node, s, next)
    setImage(node, s, next);
    setImage(next, node!.data.invr[s], node);
end;

* Carefully setting $x.s$ to $y$ means 
  * checking if $x.s$ is already defined; if not, set $x.s$ to $y$.
  * Otherwise, with $x.s = z$, say, check if $y = z$:  if so there is nothing to do.
  * Otherwise, set $z = y$ (or $y = z$ depending on which came first) and live with the consequenses ...

In [ ]:
setImage := function(node, s, next)
    local   pair;
    if IsBound(node!.next[s]) then
        pair := Set(List([next, node!.next[s]], flat));
        if Length(pair) = 2 then           # coincidence: stack!
            mergeNodes(pair[2], pair[1]);
        fi;
    else
        node!.next[s] := next;           # deduction!
    fi;
end;

* to merge nodes $z$ and $y$:
  * set $z$.`flat` to $y$
  * for each $z.s \neq {\perp}$, carefully update $z.s = y.s$.

In [ ]:
mergeNodes := function(node, other)
    local   s;
    node!.flat := other;
    node!.data.active := node!.data.active - 1;
    for s in PositionsBound(node!.next) do
        updateEdge(other, s, node!.next[s]);
    od;
end;

In [ ]:
G := G12;

In [ ]:
data := enumerate(G);

In [ ]:
nodes := Filtered(data.list, isActive);

In [ ]:
Sortex(List(nodes, x -> flat(x!.next[1])!.idx)); #:-)

In [ ]:
gens := List(G.gens, i -> Sortex(List(nodes, x -> flat(x!.next[i])!.idx)));

In [ ]:
sizeOfGroup(GroupWithGenerators(gens));

* Next: carefully extract and plot the graph (on active nodes only)!

In [ ]:
edges := Union(List(nodes, node -> List(node!.next, x -> [node!.idx, flat(x)!.idx])));;
edges := Filtered(edges, x -> x[1] <> x[2]);

In [ ]:
PlotGraph(edges, opts);

##  Exercises, etc.

* ($*$) Modify the coset enumeration procedure so that it applies to monoid presentations.

* Find matrices for the reflection representation of a given complex reflection group $G$.

* Find a way to enumerate the (conjugacy classes of) parabolic subgroups $P$ of a complex reflection group $G$.

* Compute the normalizer of parabolic subgroup $P$ in $G$.  Does $P$ always have a complement $H$? If so, does $H$ have a natural set of generators? 

* ...